### DATA MAPPING


In [3]:
import pandas as pd
import numpy as np
from datetime import datetime
import pytz

# Function to convert ISO dates to Taipei time and format properly
def convert_date(date_str, format_type):
    if pd.isna(date_str) or date_str in ['', 'None']:
        return ''
    
    taipei_tz = pytz.timezone('Asia/Taipei')
    
    try:
        # Handle ISO format (with 'T' and 'Z')
        if 'T' in date_str and date_str.endswith('Z'):
            dt = datetime.fromisoformat(date_str.replace('Z', '+00:00'))
            dt_taipei = dt.astimezone(taipei_tz)
            
            if format_type == 'acquisition_date':
                return dt_taipei.strftime('%m/%d/%Y')
            else:  # full timestamp
                return dt_taipei.strftime('%m/%d/%Y %I:%M:%S %p').replace(' AM', ' AM').replace(' PM', ' PM')
        
        # Handle existing date formats (like '04/22/2008')
        elif '/' in date_str:
            if format_type == 'acquisition_date':
                return date_str
            else:
                # Parse existing date and add time if needed
                dt = datetime.strptime(date_str, '%m/%d/%Y')
                return dt.strftime('%m/%d/%Y %I:%M:%S %p').replace(' AM', ' AM').replace(' PM', ' PM')
        
        # Fallback to return original if format not recognized
        return date_str
    
    except Exception as e:
        print(f"Error converting date: {date_str} - {str(e)}")
        return date_str

# Load mapping data
mapping_df = pd.read_excel('../proddata/mapping.xlsx', sheet_name='Inventory_Items', header=None, nrows=2)
output_headers = mapping_df.iloc[0].tolist()
source_columns = mapping_df.iloc[1].tolist()

# Clean source columns
source_columns = [str(x).strip() if not pd.isna(x) else '' for x in source_columns]

# Load source data
source_df = pd.read_csv('../proddata/source.csv')
source_df.columns = [col.strip() for col in source_df.columns]

# Create output DataFrame
output_df = pd.DataFrame(columns=output_headers)

# Process each column in mapping
for i, out_col in enumerate(output_headers):
    src_col = source_columns[i]
    
    if not src_col or src_col not in source_df.columns:
        output_df[out_col] = ''
        continue
        
    # Handle date conversions
    if out_col == 'Acquisition Date':
        output_df[out_col] = source_df[src_col].apply(lambda x: convert_date(x, 'acquisition_date'))
    elif out_col in ['Date Created', 'Date Modified']:
        output_df[out_col] = source_df[src_col].apply(lambda x: convert_date(x, 'timestamp'))
    else:
        output_df[out_col] = source_df[src_col]

# Save to Excel
output_df.to_excel('../output/formatted_inventory.xlsx', index=False)
print("Processing complete! Output saved to 'formatted_inventory.xlsx'")

Processing complete! Output saved to 'formatted_inventory.xlsx'
